In [ ]:
!wget https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_train_val.txt
!wget https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_test.txt

--2021-10-08 10:01:18--  https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_train_val.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2661060 (2.5M) [text/plain]
Saving to: ‘fci_train_val.txt.1’

fci_train_val.txt.1 100%[===================>]   2.54M  --.-KB/s    in 0.07s   

2021-10-08 10:01:18 (37.1 MB/s) - ‘fci_train_val.txt.1’ saved [2661060/2661060]

--2021-10-08 10:01:18--  https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response

In [ ]:
def read_data(filename):

    with open(filename, "r", encoding="UTF8") as f:
        data = [line.split("\t") for line in f.read().splitlines()]

    return data

train_data = read_data("/content/fci_train_val.txt")
X_train_data = [t[1] for t in train_data]
y_train_data = [int(t[0]) for t in train_data]

test_data = read_data("/content/fci_test.txt")
X_test_data = [t[1] for t in test_data]
y_test_data = [int(t[0]) for t in test_data]

In [ ]:
print(len(X_train_data))
print(len(y_train_data))
print(len(X_test_data))
print(len(y_test_data))

55134
55134
6121
6121


In [ ]:
!pip install konlpy

import nltk
nltk.download('punkt')

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 52.8 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import random

def swap(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap(new_words)
	return new_words

def EDA(sentence, alpha_rs=0.1, num_aug=4):
    words = sentence.split(' ')
    num_words = len(words)   
    augmented_sentences = []
    num_new_per_technique = int(num_aug/4) + 1
    n_rs = max(1, int(alpha_rs*num_words))

    for x in range(num_new_per_technique): 
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(" ".join(a_words))

    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    
    augmented_sentences.append(sentence)   
    return augmented_sentences
 
aug = []
for sentence in X_train_data : 
    augmented_sentences = EDA(sentence, alpha_rs = 0.1, num_aug = 4)
    aug.append(augmented_sentences)

X_train_aug = []
y_train_aug = []

for sentence in aug :  
    index = aug.index(sentence)
    for aug_sentence in sentence : 
        X_train_aug.append(aug_sentence)
        y_train_aug.append(y_train_data[index])
    
print(len(X_train_aug)) 
print(len(y_train_aug))

## result x_trian_aug 165402 
##        y_train_aug 165402

165402
165402


In [ ]:
# 형태소 분리

import numpy as np
import nltk
from konlpy.tag import Okt
pos_tagger = Okt()

def twit_token(doc):
    x = [t[0] for t in pos_tagger.pos(doc)]
    return ' '.join(x)

len_train = int(np.floor(len(X_train_aug)*0.9))
fci_data_train  = X_train_aug[:len_train]
fci_data_test   = X_train_aug[len_train:]
fci_label_train = y_train_aug[:len_train]
fci_label_test  = y_train_aug[len_train:]

fci_token_train = [twit_token(row) for row in fci_data_train]
fci_token_test  = [twit_token(row) for row in fci_data_test]

fci_sp_token_train = [nltk.word_tokenize(row) for row in fci_token_train]
fci_sp_token_test  = [nltk.word_tokenize(row) for row in fci_token_test]

In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 3.3 MB/s 
  Using cached pybind11-2.8.0-py2.py3-none-any.whl (207 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3119389 sha256=3f5399e8cc3b9274f78aea3f15d12f229c04cfae95da491eba3fb3be21cf790a
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
!gdown --id 1jHbjOcnaLourFzNuP47yGQVhBTq6Wgor --output vector.zip

Downloading...
From: https://drive.google.com/uc?id=1jHbjOcnaLourFzNuP47yGQVhBTq6Wgor
To: /content/vector.zip
100% 874M/874M [00:06<00:00, 136MB/s]


In [ ]:
!unzip -qq "/content/vector.zip"

replace model_drama.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [ ]:
#import fasttext
#model_ft = fasttext.load_model('content/model_drama.bin')
# In case fasttext is not installed
# use FastText wrapper in Gensim as:
from gensim.models.wrappers import FastText
model_ft = FastText.load_fasttext_format('model_drama.bin')


In [ ]:
def featurize_cnn(corpus,wdim,maxlen):
    conv_total = np.zeros((len(corpus),maxlen,wdim,1))
    for i in range(len(corpus)):
        if i%1000 ==0:
            print(i)
        s = corpus[i]
        for j in range(len(s)):
            if s[-j-1] in model_ft and j < maxlen:
                conv_total[i][-j-1,:,0] = model_ft[s[-j-1]]
    return conv_total

In [ ]:
fci_conv = featurize_cnn(fci_sp_token_train+fci_sp_token_test,100,30)

tfidf로 시도했던 내용

In [ ]:
# Referred to the followings for the code:
# https://gist.github.com/jason-riddle/1a854af26562c0cdb1e6ff550d1bf32d#file-complex-tf-idf-example-py-L40
# http://blog.christianperone.com/2011/10/machine-learning-text-feature-extraction-tf-idf-part-ii/

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf_vectorizer 함수는 train corpus에서 tf-idf statistics를 뽑아내고 이를 이용해 train corpus+test corpus를 수치화하는 과정
def tfidf_featurizer(corpus_train,corpus_test):
    count_vectorizer = CountVectorizer()
    count_vectorizer.fit_transform(corpus_train)
    freq_term_matrix = count_vectorizer.transform(corpus_train)
    tfidf = TfidfTransformer(norm="l2")
    tfidf.fit(freq_term_matrix)

    # unigram features
    tfidf_token = TfidfVectorizer(ngram_range=(1,1),max_features=3000)
    token_tfidf_total = tfidf_token.fit_transform(corpus_train+corpus_test)
    token_tfidf_total_mat = token_tfidf_total.toarray()

    # bigram features
    tfidf_bi_token = TfidfVectorizer(ngram_range=(1,2),max_features=3000)
    token_tfidf_bi_total = tfidf_bi_token.fit_transform(corpus_train+corpus_test)
    token_tfidf_bi_total_mat = token_tfidf_bi_total.toarray()
    return token_tfidf_total_mat,token_tfidf_bi_total_mat

fci_tfidf,fci_tfidf_bi  = tfidf_featurizer(fci_token_train,fci_token_test)
fci_tfidf_train    = fci_tfidf[:len_train]
fci_tfidf_test     = fci_tfidf[len_train:]
fci_tfidf_bi_train = fci_tfidf_bi[:len_train]
fci_tfidf_bi_test  = fci_tfidf_bi[len_train:]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

classifier_uni = LogisticRegression(random_state=1234)
classifier_uni.fit(fci_tfidf_train,fci_label_train)
uni_pred = classifier_uni.predict(fci_tfidf_test)

accuracy_score(uni_pred,fci_label_test)
metrics.f1_score(uni_pred,fci_label_test,average="macro")
metrics.f1_score(uni_pred,fci_label_test,average="weighted")
precision_recall_fscore_support(uni_pred,fci_label_test)

classifier_bi = LogisticRegression(random_state=1234)
classifier_bi.fit(fci_tfidf_bi_train,fci_label_train)
bi_pred = classifier_bi.predict(fci_tfidf_test)

accuracy_score(bi_pred,fci_label_test)
metrics.f1_score(bi_pred,fci_label_test,average="macro")
metrics.f1_score(bi_pred,fci_label_test,average="weighted")
precision_recall_fscore_support(bi_pred,fci_label_test)

1st try result

acc: 0.7665941240478781

f1_score_macro: 0.621190123136072

f1_score_weighted: 0.7752608151283553


2nd try result

acc: 0.7680309533885497

f1_score_macro: 0.6391839862416706

f1_score_weighted: 0.7741350836238543